In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
adata = sc.read_h5ad("../data/adata/tgfb.h5ad")

In [ ]:
sc.experimental.pp.highly_variable_genes(
    adata, flavor="pearson_residuals", n_top_genes=500
)

adata.layers["raw"] = adata.X.copy()
adata.layers["sqrt_norm"] = np.sqrt(sc.pp.normalize_total(adata, inplace=False)["X"])

In [ ]:
sc.experimental.pp.normalize_pearson_residuals(adata)

In [ ]:
subtype = "Cd8_T-Cell_P14"

adata_subset = adata[(adata.obs["Subtype"] == subtype)].copy()

In [ ]:
sc.tl.rank_genes_groups(
    adata_subset, groupby="batch", key_added="rank_genes_roll", method="wilcoxon"
)
sc.pl.rank_genes_groups(adata_subset, key="rank_genes_roll", groups=["WT", "KO"])

In [ ]:
def get_top_genes(adata, group, n, key="rank_genes_roll"):
    res = sc.get.rank_genes_groups_df(adata_subset, key=key, group=group)
    return res["names"].to_list()[0:n]


gene_list = [
    *get_top_genes(adata_subset, "WT", 4),
    *get_top_genes(adata_subset, "KO", 4),
]

gene_list

In [ ]:
sc.pl.dotplot(
    adata_subset,
    var_names=gene_list,
    groupby="batch",
    use_raw=False,
    dendrogram=False,  # Set to True if you want to include dendrogram
    color_map="viridis",  # Set the colormap as needed
    dot_max=0.7,  # Adjust dot size
)